#  企业偷税漏税自动检测

## 目标：根据采集到销售店铺的数据，智能识别偷税行为。

In [1]:
import pandas as pd
data =pd.read_excel('偷漏税数据.xls')

In [2]:
data.head()

,纳税人编号,销售类型,销售模式,汽车销售平均毛利,维修毛利,企业维修收入占销售收入比重,增值税税负,存货周转率,成本费用利润率,整体理论税负,整体税负控制数,办牌率,单台办牌手续费收入,代办保险率,保费返还率,输出
0,1,国产轿车,4S店,0.0635,0.3241,0.0879,0.0084,8.5241,0.0018,0.0166,0.0147,0.4000,0.02,0.7155,0.1500,正常
1,2,国产轿车,4S店,0.0520,0.2577,0.1394,0.0298,5.2782,-0.0013,0.0032,0.0137,0.3307,0.02,0.2697,0.1367,正常
2,3,国产轿车,4S店,0.0173,0.1965,0.1025,0.0067,19.8356,0.0014,0.0080,0.0061,0.2256,0.02,0.2445,0.1301,正常
3,4,国产轿车,一级代理商,0.0501,0.0000,0.0000,0.0000,1.0673,-0.3596,-0.1673,0.0000,0.0000,0.00,0.0000,0.0000,异常
4,5,进口轿车,4S店,0.0564,0.0034,0.0066,0.0017,12.8470,-0.0014,0.0123,0.0095,0.0039,0.08,0.0117,0.1872,正常


### 经观察数据，筛选空值数据 ，并对比两类数据的差异

In [3]:
data=data[(data['汽车销售平均毛利']!=0)| (data['维修毛利']!=0)| (data['企业维修收入占销售收入比重']!=0)| (data['增值税税负']!=0)|(data['存货周转率']!=0)]

In [4]:
columns=data.describe().columns

In [5]:
data_true=data[data['输出']=='正常']
data_false=data[data['输出']=='异常']

In [6]:
data2=pd.concat([data_true.describe(),data_false.describe()],axis=1)

In [7]:
columns

Index(['纳税人编号', '汽车销售平均毛利', '维修毛利', '企业维修收入占销售收入比重', '增值税税负', '存货周转率',
       '成本费用利润率', '整体理论税负', '整体税负控制数', '办牌率', '单台办牌手续费收入', '代办保险率', '保费返还率'],
      dtype='object')

In [11]:
data2[columns].T['mean']

纳税人编号            53.614286
纳税人编号            68.953488
汽车销售平均毛利          0.039199
汽车销售平均毛利          0.004558
维修毛利              0.308890
维修毛利             -0.056174
企业维修收入占销售收入比重     0.088906
企业维修收入占销售收入比重     0.053430
增值税税负             0.008576
增值税税负             0.009937
存货周转率            10.529039
存货周转率            14.686005
成本费用利润率           0.179729
成本费用利润率           0.234860
整体理论税负            0.011963
整体理论税负            0.010616
整体税负控制数           0.011541
整体税负控制数           0.001286
办牌率               0.155729
办牌率               0.167733
单台办牌手续费收入         0.016450
单台办牌手续费收入         0.020477
代办保险率             0.244171
代办保险率             0.092674
保费返还率             0.055056
保费返还率             0.023316
Name: mean, dtype: float64

In [10]:
len(data_true),len(data_false)

(70, 43)

### 根据正常和异常的样本进行描述对比，特征：‘汽车销售毛利’，‘维修毛利’，'企业维修收入占销售收入比重','存货周转率‘,'成本费用利润率’'代办保险 率','整体税负控制数 ','保费返还率'具有显著性差异，因此具体考虑以上几个特征

In [11]:
data2[['成本费用利润率', '整体理论税负']]

,成本费用利润率,成本费用利润率,整体理论税负,整体理论税负
count,70.000000,43.000000,70.000000,43.000000
mean,0.179729,0.234860,0.011963,0.010616
std,0.866830,1.551397,0.011765,0.053682
min,-0.315900,-1.000000,-0.014300,-0.181000
25%,-0.001000,-0.083700,0.006325,0.000350
50%,0.001400,0.000000,0.011750,0.007500
75%,0.018300,0.005950,0.016375,0.017950
max,6.965100,9.827200,0.068000,0.159300


In [12]:
columns

Index(['纳税人编号', '汽车销售平均毛利', '维修毛利', '企业维修收入占销售收入比重', '增值税税负', '存货周转率',
       '成本费用利润率', '整体理论税负', '整体税负控制数', '办牌率', '单台办牌手续费收入', '代办保险率', '保费返还率'],
      dtype='object')

In [13]:
data_feature=data[['汽车销售平均毛利','维修毛利','企业维修收入占销售收入比重','存货周转率','成本费用利润率','整体税负控制数','办牌率','单台办牌手续费收入','代办保险率','保费返还率']]

# 归一化&编码

In [14]:
x_train=(data_feature-data_feature.mean(axis=0))/data_feature.std()

In [15]:
y_train =data['输出']
#y_train =pd.get_dummies(y_train)  
y_train=y_train.apply(lambda x : 1  if x=='正常' else 0 )

In [16]:
y_train

0      1
1      1
2      1
3      0
4      1
5      1
6      1
7      0
8      0
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     0
20     0
21     0
22     0
23     1
24     0
25     0
26     0
27     1
28     1
29     1
      ..
87     0
88     0
90     0
92     0
93     1
94     1
95     1
96     1
97     0
98     1
99     1
100    0
101    0
102    1
103    1
104    0
106    1
107    1
108    1
109    0
110    1
113    0
114    1
116    0
117    0
118    0
120    0
121    0
122    1
123    0
Name: 输出, Length: 113, dtype: int64

In [17]:
# 划分数据集：
from sklearn.model_selection import train_test_split
X_train ,X_test,y_train,y_test =train_test_split(x_train,y_train,test_size=0.3)

# 调用决策树

In [18]:
from sklearn.tree import DecisionTreeClassifier
tree= DecisionTreeClassifier(criterion ='gini')
tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [19]:
predict=tree.predict(X_test)

In [20]:
#构建混淆矩阵
from sklearn.metrics import accuracy_score ,confusion_matrix  ,roc_auc_score
confusion_matrix (y_test,predict)

array([[ 7,  5],
       [ 0, 22]], dtype=int64)

In [21]:
accuracy_score(y_test,predict)

0.8529411764705882